# Análisis del Padrón Municipal de Madrid

La población de un municipio se define como el conjunto de personas inscritas en su Padrón municipal.

Toda persona que viva en España está obligada a inscribirse en el Padrón del municipio en el que resida habitualmente, inscribiéndose en el que habite durante más tiempo al año si vive en varios municipios. 

El Padrón Municipal de Habitantes es el registro administrativo donde constan los habitantes de un municipio, siendo sus datos prueba de la residencia en el municipio y del domicilio habitual.

En el [Portal de datos abiertos del Ayuntamiento de Madrid](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=1d755cde99be2410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default) podemos encontrar información del Padrón municipal, detallado a nivel de distrito, barrio y sección censal, y agregado por sexo y edad.

Con esta información, vamos a tratar de responder a las siguientes cuestiones:

1. ¿Cuál es el porcentaje de españoles y extranjeros que hay por distrito y barrio?
2. ¿Cuál es la edad media de los ciudadanos por distrito y barrio?

### Instalación de Spark en Google Colab

In [1]:
# Get the latest Spark version
from bs4 import BeautifulSoup
import requests

url = 'https://downloads.apache.org/spark/' 
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)

link_files = []
for link in soup.find_all('a'):
  link_files.append(link.get('href'))
spark_link = [x for x in link_files if 'spark' in x]  
spark_version = spark_link[1][:-1]

In [2]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install needed libraries
!pip install -q findspark
!pip install -q pyspark
!pip install -q koalas
#!pip install -q pyarrow
!pip install --upgrade pyarrow
!pip install -q pyngrok

# Download Spark package
import os
dl_link = f"https://www-us.apache.org/dist/spark/{spark_version}/{spark_version}-bin-hadoop2.7.tgz"
os.system(f"wget -q {dl_link}")

# Unzip the spark file to the current folder
os.system(f"tar xf {spark_version}-bin-hadoop2.7.tgz")

# Download package to create a public url to view the Spark UI page
#!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip

     |████████████████████████████████| 204.8MB 62kB/s 
     |████████████████████████████████| 204kB 47.9MB/s 
     |████████████████████████████████| 675kB 5.5MB/s 
     |████████████████████████████████| 20.7MB 1.2MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 737kB 4.3MB/s 


0

In [3]:
# Set your spark folder to your system path environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

# Locate Spark in the system
import findspark
findspark.init()

### Importar datos desde Google Drive

In [8]:
# Mount Drive (authentication required)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Viewing the data in the folder present from the drive
!ls -l "/content/drive/My Drive/Colab Notebooks/datasets"

In [29]:
# Cloud file path
file_path = "drive/MyDrive/Colab\ Notebooks/datasets/padron_madrid_2020.csv"

### Importar datos desde local

In [ ]:
# Importing files method from colab for accessing Local file system
from google.colab import files

uploaded = files.upload()

print(uploaded.keys())

In [2]:
# Local file path
file_path = './Documents/Learning Spark/datasets/padron_madrid_2020.csv'

## Análisis usando PySpark

In [4]:
# Import PySpark and libraries
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
#from pyspark.sql.types import *
from pyngrok import ngrok

In [5]:
# Build a SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Padrón")\
        .config("spark.sql.execution.arrow.enabled", "true")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Open tunnel on the port 4050 to get a public URL
ngrok.connect(4050)

In [6]:
# Print the SparkSession
spark

In [7]:
# Define schema for our data using the StructType method
schema = StructType([
  # Define a StructField for each field
    StructField('cod_distrito', StringType(), False),
    StructField('desc_distrito', StringType(), False),
    StructField('cod_dist_barrio', StringType(), False),
    StructField('desc_barrio', StringType(), False),
    StructField('cod_barrio', StringType(), False),
    StructField('cod_dist_seccion', StringType(), False),
    StructField('cod_seccion', StringType(), False),
    StructField('cod_edad', IntegerType(), False),
    StructField('esp_hombres', IntegerType(), True),
    StructField('esp_mujeres', IntegerType(), True),
    StructField('ext_hombres', IntegerType(), True),
    StructField('ext_mujeres', IntegerType(), True)
])

# Define schema for our data using DDL (Data Definition Language)
schema_ddl = """
  cod_distrito STRING, 
  desc_distrito STRING, 
  cod_dist_barrio STRING, 
  desc_barrio STRING,
  cod_barrio STRING, 
  cod_dist_seccion STRING, 
  cod_seccion STRING, 
  cod_edad INT, 
  esp_hombres INT, 
  esp_mujeres INT, 
  ext_hombres INT, 
  ext_mujeres INT
"""

In [8]:
padron_raw = (spark.read.format('csv')
            .options(header=True, delimiter=';',
                     encoding='ISO-8859-1',
                     emptyValue=0,
                     ignoreLeadingWhiteSpace=True,
                     ignoreTrailingWhiteSpace=True
                    )
            .schema(schema)
            .load(file_path))

In [38]:
padron_raw.printSchema()

# Count the number of rows 
print("There are {} rows in the DataFrame. \n".format(padron_raw.count()))

root
 |-- cod_distrito: string (nullable = true)
 |-- desc_distrito: string (nullable = true)
 |-- cod_dist_barrio: string (nullable = true)
 |-- desc_barrio: string (nullable = true)
 |-- cod_barrio: string (nullable = true)
 |-- cod_dist_seccion: string (nullable = true)
 |-- cod_seccion: string (nullable = true)
 |-- cod_edad: integer (nullable = true)
 |-- esp_hombres: integer (nullable = true)
 |-- esp_mujeres: integer (nullable = true)
 |-- ext_hombres: integer (nullable = true)
 |-- ext_mujeres: integer (nullable = true)

There are 237675 rows in the DataFrame. 



In [37]:
# Show the 10 first rows
padron_raw.show(10)

+------------+--------------------+---------------+--------------------+----------+----------------+-----------+--------+-----------+-----------+-----------+-----------+
|cod_distrito|       desc_distrito|cod_dist_barrio|         desc_barrio|cod_barrio|cod_dist_seccion|cod_seccion|cod_edad|esp_hombres|esp_mujeres|ext_hombres|ext_mujeres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+--------+-----------+-----------+-----------+-----------+
|           1|CENTRO              |            101|PALACIO             |         1|            1006|          6|     103|          0|          1|          0|          0|
|           1|CENTRO              |            101|PALACIO             |         1|            1007|          7|       0|          1|          1|          0|          3|
|           1|CENTRO              |            101|PALACIO             |         1|            1007|          7|       1|          2|          3|     

In [39]:
# Get count of null values in Pyspark
padron_raw.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in padron_raw.columns[:8]]).show()

padron_raw.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in padron_raw.columns[8:]]).show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+--------+
|cod_distrito|desc_distrito|cod_dist_barrio|desc_barrio|cod_barrio|cod_dist_seccion|cod_seccion|cod_edad|
+------------+-------------+---------------+-----------+----------+----------------+-----------+--------+
|           0|            0|              0|          0|         0|               0|          0|       0|
+------------+-------------+---------------+-----------+----------+----------------+-----------+--------+

+-----------+-----------+-----------+-----------+
|esp_hombres|esp_mujeres|ext_hombres|ext_mujeres|
+-----------+-----------+-----------+-----------+
|          0|          0|          0|          0|
+-----------+-----------+-----------+-----------+



In [ ]:
#[f.name for f in padron_raw.schema.fields if isinstance(f.dataType, StringType)]

In [ ]:
#map(padronDF.schema.fields)

In [ ]:
#padronDF.select(map()
#for f in padronDF.schema.fields:
#  if(f.dataType == StringType()):
#    F.trim(F.col(f.name))

In [40]:
padron_raw.filter(F.col("desc_distrito") == "CENTRO").show(3)

+------------+-------------+---------------+-----------+----------+----------------+-----------+--------+-----------+-----------+-----------+-----------+
|cod_distrito|desc_distrito|cod_dist_barrio|desc_barrio|cod_barrio|cod_dist_seccion|cod_seccion|cod_edad|esp_hombres|esp_mujeres|ext_hombres|ext_mujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+--------+-----------+-----------+-----------+-----------+
+------------+-------------+---------------+-----------+----------+----------------+-----------+--------+-----------+-----------+-----------+-----------+



In [41]:
padronDF = padron_raw.select(
  F.trim(F.col("desc_distrito")).alias("distrito"),
  F.trim(F.col("desc_barrio")).alias("barrio"),
  "cod_edad","esp_hombres", "esp_mujeres", "ext_hombres", "ext_mujeres")

In [42]:
padronDF.show(10)

+--------+-------+--------+-----------+-----------+-----------+-----------+
|distrito| barrio|cod_edad|esp_hombres|esp_mujeres|ext_hombres|ext_mujeres|
+--------+-------+--------+-----------+-----------+-----------+-----------+
|  CENTRO|PALACIO|     103|          0|          1|          0|          0|
|  CENTRO|PALACIO|       0|          1|          1|          0|          3|
|  CENTRO|PALACIO|       1|          2|          3|          0|          0|
|  CENTRO|PALACIO|       2|          1|          4|          0|          0|
|  CENTRO|PALACIO|       3|          4|          0|          0|          0|
|  CENTRO|PALACIO|       4|          1|          2|          0|          1|
|  CENTRO|PALACIO|       5|          2|          6|          0|          0|
|  CENTRO|PALACIO|       6|          1|          0|          0|          0|
|  CENTRO|PALACIO|       7|          3|          2|          0|          0|
|  CENTRO|PALACIO|       8|          4|          2|          0|          0|
+--------+--

In [43]:
padronDF.filter(F.col("distrito") == "CENTRO").show(3)

+--------+-------+--------+-----------+-----------+-----------+-----------+
|distrito| barrio|cod_edad|esp_hombres|esp_mujeres|ext_hombres|ext_mujeres|
+--------+-------+--------+-----------+-----------+-----------+-----------+
|  CENTRO|PALACIO|     103|          0|          1|          0|          0|
|  CENTRO|PALACIO|       0|          1|          1|          0|          3|
|  CENTRO|PALACIO|       1|          2|          3|          0|          0|
+--------+-------+--------+-----------+-----------+-----------+-----------+
only showing top 3 rows



### ¿Qué porcentaje de españoles y extranjeros hay por distrito y barrio?

In [44]:
padronDF = (padronDF
                .withColumn('total_esp', F.col('esp_hombres') + F.col('esp_mujeres'))
                .withColumn('total_ext', F.col('ext_hombres') + F.col('ext_mujeres'))
                .withColumn('total', F.col('total_esp') + F.col('total_ext'))
               )

padronDF.show(10)

+--------+-------+--------+-----------+-----------+-----------+-----------+---------+---------+-----+
|distrito| barrio|cod_edad|esp_hombres|esp_mujeres|ext_hombres|ext_mujeres|total_esp|total_ext|total|
+--------+-------+--------+-----------+-----------+-----------+-----------+---------+---------+-----+
|  CENTRO|PALACIO|     103|          0|          1|          0|          0|        1|        0|    1|
|  CENTRO|PALACIO|       0|          1|          1|          0|          3|        2|        3|    5|
|  CENTRO|PALACIO|       1|          2|          3|          0|          0|        5|        0|    5|
|  CENTRO|PALACIO|       2|          1|          4|          0|          0|        5|        0|    5|
|  CENTRO|PALACIO|       3|          4|          0|          0|          0|        4|        0|    4|
|  CENTRO|PALACIO|       4|          1|          2|          0|          1|        3|        1|    4|
|  CENTRO|PALACIO|       5|          2|          6|          0|          0|       

In [45]:
padronDF_grouped = padronDF.groupby('distrito','barrio')\
    .agg(
            (F.round(100 * F.sum('total_esp') / F.sum('total'), 2)).alias('pct_esp'),
            (F.round(100 * F.sum('total_ext') / F.sum('total'), 2)).alias('pct_ext')
    )\
    .orderBy('pct_ext', ascending=False)

padronDF_grouped.show(10)

+------------------+-------------+-------+-------+
|          distrito|       barrio|pct_esp|pct_ext|
+------------------+-------------+-------+-------+
|             USERA|   PRADOLONGO|  62.16|  37.84|
|        VILLAVERDE|SAN CRISTOBAL|  63.52|  36.48|
|PUENTE DE VALLECAS|    SAN DIEGO|  67.14|  32.86|
|            CENTRO|          SOL|  68.02|  31.98|
|             USERA|     MOSCARDO|  70.23|  29.77|
|            CENTRO|  EMBAJADORES|   70.5|   29.5|
|             USERA|  ALMENDRALES|  70.64|  29.36|
|             USERA|        ZOFIO|  72.95|  27.05|
|            CENTRO|       CORTES|  74.14|  25.86|
|       CARABANCHEL|PUERTA BONITA|  74.34|  25.66|
+------------------+-------------+-------+-------+
only showing top 10 rows



### ¿Cuál es la edad media de los ciudadanos por barrio?

In [47]:
(padronDF
 .select("barrio", "cod_edad")
 .groupBy("barrio")
 .agg((F.round(F.avg("cod_edad"),2)).alias("avg_edad"))
 .orderBy("avg_edad", ascending=False)
 .show(10)
)

+--------------+--------+
|        barrio|avg_edad|
+--------------+--------+
|    EL PLANTIO|   50.05|
|HISPANOAMERICA|    49.7|
|    GAZTAMBIDE|   49.52|
|       ALMAGRO|   49.49|
|    RIOS ROSAS|   49.45|
|  VALLEHERMOSO|   49.42|
| CASA DE CAMPO|   49.39|
|    CONCEPCION|   49.38|
|         IBIZA|    49.3|
|     ARGUELLES|    49.2|
+--------------+--------+
only showing top 10 rows



### Otros ejercicios

#### 6.10
Lanza una consulta contra el DF resultante en la que muestre el número total de "espanoleshombres", "espanolesmujeres", extranjeroshombres" y "extranjerosmujeres" para cada barrio de cada distrito. Las columnas distrito y barrio deben ser las primeras en aparecer en el show. Los resultados deben estar ordenados en orden de más a menos según la columna "extranjerosmujeres" y desempatarán por la columna "extranjeroshombres".

In [ ]:
padronDF.groupBy('distrito','barrio')\
    .agg(
         (F.sum("esp_hombres")).alias("esp_hombres"), 
         (F.sum("esp_mujeres")).alias("esp_mujeres"),
         (F.sum("ext_hombres")).alias("ext_hombres"),
         (F.sum("ext_mujeres")).alias("ext_mujeres")
     )\
    .orderBy("ext_mujeres", "ext_hombres", ascending=False)\
    .show()

#### 6.12

Crea un nuevo DataFrame a partir del original que muestre únicamente una columna con DESC_BARRIO, otra con DESC_DISTRITO y otra con el número total de "espanoleshombres" residentes en cada distrito de cada barrio. Únelo (con un join) con el DataFrame original a través de las columnas en común.

In [ ]:
df1 = padronDF.groupby("distrito", "barrio")\
  .agg(F.sum("esp_hombres").alias("total_esp_hombres"))

padronDF.join(df1, ["distrito","barrio"]).show(10)

#https://stackoverflow.com/questions/46944493/removing-duplicate-columns-after-a-df-join-in-spark

+--------+-------+--------+-----------+-----------+-----------+-----------+-----------------+
distrito| barrio|cod_edad|esp_hombres|esp_mujeres|ext_hombres|ext_mujeres|total_esp_hombres|
+--------+-------+--------+-----------+-----------+-----------+-----------+-----------------+
 CENTRO|PALACIO| 103| 0| 1| 0| 0| 9380|
 CENTRO|PALACIO| 0| 1| 1| 0| 3| 9380|
 CENTRO|PALACIO| 1| 2| 3| 0| 0| 9380|
 CENTRO|PALACIO| 2| 1| 4| 0| 0| 9380|
 CENTRO|PALACIO| 3| 4| 0| 0| 0| 9380|
 CENTRO|PALACIO| 4| 1| 2| 0| 1| 9380|
 CENTRO|PALACIO| 5| 2| 6| 0| 0| 9380|
 CENTRO|PALACIO| 6| 1| 0| 0| 0| 9380|
 CENTRO|PALACIO| 7| 3| 2| 0| 0| 9380|
 CENTRO|PALACIO| 8| 4| 2| 0| 0| 9380|
+--------+-------+--------+-----------+-----------+-----------+-----------+-----------------+
only showing top 10 rows

#### 6.13 (pending)
Repite la función anterior utilizando funciones de ventana. (over(Window.partitionBy.....)).

from pyspark.sql.window import Window
Defines partitioning specification and ordering specification.
windowSpec = \
  Window \
    .partitionBy(padronDF.distrito, padronDF.barrio) \
    .orderBy(...)

#### 6.14
Mediante una función Pivot muestra una tabla (que va a ser una tabla de contingencia) que contenga los valores medios de `espanolesmujeres` para cada barrio y en cada rango de edad (COD_EDAD_INT). Los barrios incluidos deben ser únicamente CENTRO, BARAJAS y RETIRO y deben figurar como columnas.

In [48]:
padronDF.filter("""distrito in ("CENTRO", "BARAJAS", "RETIRO")""")\
  .groupBy("cod_edad", "distrito")\
  .mean("esp_mujeres")\
  .orderBy("cod_edad")\
  .show(10)

+--------+--------+------------------+
|cod_edad|distrito|  avg(esp_mujeres)|
+--------+--------+------------------+
|       0|  RETIRO|3.4193548387096775|
|       0|  CENTRO|2.3545454545454545|
|       0| BARAJAS| 5.483870967741935|
|       1|  CENTRO|2.3423423423423424|
|       1| BARAJAS| 5.774193548387097|
|       1|  RETIRO|3.9361702127659575|
|       2|  CENTRO|2.3394495412844036|
|       2|  RETIRO| 4.258064516129032|
|       2| BARAJAS| 6.741935483870968|
|       3|  CENTRO|2.2181818181818183|
+--------+--------+------------------+
only showing top 10 rows



In [ ]:
padronDF.groupBy("cod_edad")\
  .pivot("distrito", ["BARAJAS", "CENTRO", "RETIRO"])\
  .agg(F.round(F.mean("esp_mujeres"),2))\
  .orderBy("cod_edad")\
  .show(10)

#### 6.15 
Utilizando este nuevo DF, crea 3 columnas nuevas que hagan referencia a qué porcentaje de la población total de "espanolesmujeres" de cada rango de edad representa cada uno delos tres distritos. Debe estar redondeada a 2 decimales. Puedes imponerte la condición extra de no apoyarte en ninguna columna auxiliar creada para el caso.

In [ ]:
padronDF_edad = padronDF.groupBy("cod_edad")\
  .pivot("distrito", ["BARAJAS", "CENTRO", "RETIRO"])\
  .sum("esp_mujeres")\
  .orderBy("cod_edad")

padronDF_edad.show(5)

In [ ]:
padronDF_edad.select("*",
                     (F.round(100 * (F.col("BARAJAS") / (F.col("BARAJAS") + F.col("CENTRO") + F.col("RETIRO"))),2)).alias("% BARAJAS"),
                     (F.round(100* (F.col("CENTRO") / (F.col("BARAJAS") + F.col("CENTRO") + F.col("RETIRO"))),2)).alias("% CENTRO"),
                     (F.round(100* (F.col("RETIRO") / (F.col("BARAJAS") + F.col("CENTRO") + F.col("RETIRO"))),2)).alias("% RETIRO")).show(10)

#### 6.16

Guarda el archivo CSV particionado por distrito y por barrio (en ese orden) en un directorio local. Consulta el directorio para ver la estructura de los ficheros y comprueba que es la esperada.

In [56]:
%fs
ls /FileStore/datasets

/content


In [57]:
padronDF.write.partitionBy("distrito","barrio").mode("overwrite").csv("/FileStore/datasets/padron.csv", header=True)

In [ ]:
%fs 
ls /FileStore/datasets/padron.csv

In [ ]:
spark.read.csv("/FileStore/datasets/padron.csv/distrito=ARGANZUELA/barrio=ACACIAS", header=True).show(5)

## Análisis usando Koalas

In [50]:
import databricks.koalas as ks

In [ ]:
padron_raw_KDF = padron_raw.to_koalas()

print(type(padron_raw_KDF))

databricks.koalas.frame.DataFrame


In [ ]:
# Creamos una función para eliminar espacios en columnas de texto
def df_strip(df):
  ks.set_option('compute.ops_on_diff_frames', True)
  df_obj = df.select_dtypes(['object'])
  df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
  ks.set_option('compute.ops_on_diff_frames', False)
  return df
#https://koalas.readthedocs.io/en/latest/user_guide/options.html#operations-on-different-dataframes

In [ ]:
padron_raw_KDF.head()

,cod_distrito,desc_distrito,cod_dist_barrio,desc_barrio,cod_barrio,cod_dist_seccion,cod_seccion,cod_edad,esp_hombres,esp_mujeres,ext_hombres,ext_mujeres
0,1,CENTRO,101,PALACIO,1,1006,6,103,0,1,0,0
1,1,CENTRO,101,PALACIO,1,1007,7,0,1,1,0,3
2,1,CENTRO,101,PALACIO,1,1007,7,1,2,3,0,0
3,1,CENTRO,101,PALACIO,1,1007,7,2,1,4,0,0
4,1,CENTRO,101,PALACIO,1,1007,7,3,4,0,0,0


In [ ]:
padronKDF = df_strip(padron_raw_KDF)

In [ ]:
padronKDF[padronKDF['desc_distrito'] == "CENTRO"].head()

,cod_distrito,desc_distrito,cod_dist_barrio,desc_barrio,cod_barrio,cod_dist_seccion,cod_seccion,cod_edad,esp_hombres,esp_mujeres,ext_hombres,ext_mujeres
0,1,CENTRO,101,PALACIO,1,1006,6,103,0,1,0,0
1,1,CENTRO,101,PALACIO,1,1007,7,0,1,1,0,3
2,1,CENTRO,101,PALACIO,1,1007,7,1,2,3,0,0
3,1,CENTRO,101,PALACIO,1,1007,7,2,1,4,0,0
4,1,CENTRO,101,PALACIO,1,1007,7,3,4,0,0,0


In [ ]:
padronKDF = padronKDF.drop(["cod_distrito", "cod_dist_barrio"])
# no permite borrar filas axis = 0

In [ ]:
padronKDF["total_esp"] = padronKDF["esp_hombres"] + padronKDF["esp_mujeres"]
padronKDF["total_ext"] = padronKDF["ext_hombres"] + padronKDF["ext_mujeres"]

padronKDF.head()

,desc_distrito,desc_barrio,cod_barrio,cod_dist_seccion,cod_seccion,cod_edad,esp_hombres,esp_mujeres,ext_hombres,ext_mujeres,total_esp,total_ext
0,CENTRO,PALACIO,1,1006,6,103,0,1,0,0,1,0
1,CENTRO,PALACIO,1,1007,7,0,1,1,0,3,2,3
2,CENTRO,PALACIO,1,1007,7,1,2,3,0,0,5,0
3,CENTRO,PALACIO,1,1007,7,2,1,4,0,0,5,0
4,CENTRO,PALACIO,1,1007,7,3,4,0,0,0,4,0


## Análisis usando SQL

In [ ]:
# Create a temporal table from DataFrame
padronDF.createOrReplaceTempView('padron_ltv')

# Print the tables in the catalog
#print(spark.catalog.listTables())

In [ ]:
query = """
SELECT
  distrito AS DISTRITO,
  barrio AS BARRIO,
  ROUND(100 * SUM(total_esp)/SUM(total), 2) AS PCT_ESP,
  ROUND(100 * SUM(total_ext)/SUM(total), 2) AS PCT_EXT
  FROM padron_ltv
  GROUP BY distrito, barrio
  ORDER BY pct_ext DESC
  LIMIT 10
"""
spark.sql(query).show()

+------------------+-------------+-------+-------+
|          DISTRITO|       BARRIO|PCT_ESP|PCT_EXT|
+------------------+-------------+-------+-------+
|             USERA|   PRADOLONGO|  62.16|  37.84|
|        VILLAVERDE|SAN CRISTOBAL|  63.52|  36.48|
|PUENTE DE VALLECAS|    SAN DIEGO|  67.14|  32.86|
|            CENTRO|          SOL|  68.02|  31.98|
|             USERA|     MOSCARDO|  70.23|  29.77|
|            CENTRO|  EMBAJADORES|   70.5|   29.5|
|             USERA|  ALMENDRALES|  70.64|  29.36|
|             USERA|        ZOFIO|  72.95|  27.05|
|            CENTRO|       CORTES|  74.14|  25.86|
|       CARABANCHEL|PUERTA BONITA|  74.34|  25.66|
+------------------+-------------+-------+-------+



### Hive

In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS datos_padron;
USE datos_padron;
SHOW DATABASES;

In [ ]:
%sql
SHOW TABLES;

In [ ]:
%sql
DROP TABLE IF EXISTS padron_raw;
CREATE EXTERNAL TABLE datos_padron.padron_raw
(
	cod_distrito STRING,
	desc_distrito STRING,
	cod_dist_barrio STRING,
	desc_barrio STRING,
	cod_barrio STRING,
	cod_dist_seccion STRING,
	cod_seccion STRING,
	cod_edad INT,
	espanoles_hombres INT,
	espanoles_mujeres INT,
	extranjeros_hombres INT,
	extranjeros_mujeres INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES ('input.regex'='"(\\d*)";"(.*?)\\s*";"(\\d*)";"(.*?)\\s*";"(\\d*)";"(\\d*)";"(\\d*)";"(\\d*)";"(\\d*)";"(\\d*)";"(\\d*)";"(\\d*)"')
TBLPROPERTIES (
    "skip.header.line.count"="1",
    "seriealization.null.format"="0"
)
LOCATION '/FileStore/datasets/padron_madrid_2020.csv';

In [ ]:
%sql
SELECT * FROM padron_raw LIMIT 5;

In [ ]:
%sql
-- Check the number of rows
SELECT COUNT(*) AS total_rows FROM padron_raw;

In [ ]:
%sql
DROP TABLE IF EXISTS padron_clean;
CREATE TABLE datos_padron.padron_clean
AS 
SELECT
  cod_distrito,
  TRIM(desc_distrito) AS distrito,
  cod_dist_barrio,
  TRIM(desc_barrio) AS barrio,
  cod_barrio,
  cod_dist_seccion,
  cod_seccion,
  CAST(cod_edad AS INT),
  CAST(espanoles_hombres AS INT),
  CAST(espanoles_mujeres AS INT),
  CAST(extranjeros_hombres AS INT),
  CAST(extranjeros_mujeres AS INT)
FROM datos_padron.padron_raw
WHERE (cod_distrito IS NOT NULL) AND (cod_barrio IS NOT NULL);

In [ ]:
%sql
-- Check the number of rows
SELECT COUNT(*) AS total_rows FROM padron_clean;

In [ ]:
%sql
SHOW TABLES;

In [ ]:
%sql
SELECT * FROM padron_clean LIMIT 5;

In [ ]:
%sql
SELECT COUNT(*) FROM padron_clean
WHERE espanoles_hombres IS NULL

In [ ]:
%sql
WITH total AS (
SELECT
    distrito,
    barrio,
    nvl(espanoles_hombres,0) + nvl(espanoles_mujeres,0) AS total_espanoles,
    nvl(extranjeros_hombres,0) + nvl(extranjeros_mujeres,0) AS total_extranjeros
FROM padron_clean
)

SELECT
  distrito AS DISTRITO,
  barrio AS BARRIO,
  ROUND(100 * SUM(total_espanoles)/SUM(total_espanoles + total_extranjeros), 2) AS PCT_ESP,
  ROUND(100 * SUM(total_extranjeros)/SUM(total_espanoles + total_extranjeros), 2) AS PCT_EXT
  FROM total
  GROUP BY distrito, barrio
  ORDER BY pct_ext DESC
  LIMIT 10

## Referencias

* https://www.datasciencemadesimple.com/pyspark-tutorial/
* https://sparkbyexamples.com/pyspark/
* https://mungingdata.com/pyspark/